## bi-gramのtf-idfスコア

In [1]:
# 02_calculate_tfidf.py

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# --- メインの実行処理 ---
if __name__ == '__main__':
    # 1. 前処理済みのCSVファイルを読み込む
    try:
        df_preprocessed = pd.read_csv('csv/preprocessed_bigrams.csv')
    except FileNotFoundError:
        print("エラー: 'preprocessed_bigrams.csv'が見つかりません。")
        print("先に01_preprocess_bigrams.pyを実行してください。")
        exit()

    # レビューが空の行を念のため削除
    df_preprocessed.dropna(subset=['bigram_corpus'], inplace=True)

    # 2. 分析対象のデータ（コーパス）と店舗名リストを作成
    corpus = df_preprocessed['bigram_corpus'].tolist()
    shop_names = df_preprocessed['shop_name'].tolist()

    if corpus:
        print("Bi-gramのTF-IDFスコアを計算中...")
        
        # 3. TfidfVectorizerを使い、Bi-gramごとのTF-IDFを計算
        # min_df=2: 2店舗以上で出現するBi-gramのみを対象とし、レアすぎる単語ペアは無視
        vectorizer = TfidfVectorizer(min_df=2, max_df=0.9)
        tfidf_matrix = vectorizer.fit_transform(corpus)
        
        # Bi-gramのリスト（特徴量名）を取得
        feature_names = vectorizer.get_feature_names_out()

        print("\n--- 【結果】店舗ごとの特徴的なBi-gram（TF-IDFスコア上位）---\n")
        
        # 4. 各店舗ごとに、スコアの高いBi-gramを表示
        for i, shop_name in enumerate(shop_names):
            print(f"★★★ {shop_name} ★★★")
            # i番目の店舗に対応するTF-IDFスコアの行を取得
            row = tfidf_matrix.toarray()[i]
            # スコアが高い順にトップ7のインデックスを取得
            top_n_indices = row.argsort()[::-1][:7]
            
            for index in top_n_indices:
                # スコアが0より大きいものだけ表示
                if row[index] > 0:
                    # アンダースコアを「, 」に戻して見やすく表示
                    bigram_display = feature_names[index].replace('_', ', ')
                    print(f"  - {bigram_display} (スコア: {row[index]:.3f})")
            print("-" * 30)
    else:
        print("分析対象のデータがありません。")

Bi-gramのTF-IDFスコアを計算中...

--- 【結果】店舗ごとの特徴的なBi-gram（TF-IDFスコア上位）---

★★★ 1129 by Ogawa ★★★
  - レインボー, ブリッジ (スコア: 0.241)
  - デックス, 東京 (スコア: 0.210)
  - 東京, ビーチ (スコア: 0.171)
  - ハンバーグ, ランチ (スコア: 0.167)
  - ビーチ, 6 (スコア: 0.159)
  - 6, 階 (スコア: 0.148)
  - 窓際, 席 (スコア: 0.127)
------------------------------
★★★ 82 築地店 ★★★
  - 築地, 店 (スコア: 0.284)
  - 築地, 駅 (スコア: 0.244)
  - ２, 築地 (スコア: 0.171)
  - 店, hub (スコア: 0.134)
  - 東, 銀座 (スコア: 0.131)
  - メトロ, 日比谷線 (スコア: 0.103)
  - 東京, メトロ (スコア: 0.101)
------------------------------
★★★ ABURI 百貫 有明ガーデン店 ★★★
  - 回転, 寿司 (スコア: 0.611)
  - 有明, ガーデン (スコア: 0.310)
  - 百, 貫 (スコア: 0.221)
  - 炙る, 寿司 (スコア: 0.156)
  - 寿司, 屋 (スコア: 0.150)
  - 通り, 炙る (スコア: 0.143)
  - 塩, レモン (スコア: 0.112)
------------------------------
★★★ ARIAKE ARENA CAFE ★★★
  - 有明, アリーナ (スコア: 0.547)
  - アリーナ, 行く (スコア: 0.142)
  - ライブ, 有明 (スコア: 0.132)
  - アリーナ, イベント (スコア: 0.129)
  - 参加, する (スコア: 0.125)
  - 日,  (スコア: 0.108)
  - ライブ, 前 (スコア: 0.087)
------------------------------
★★★ ARIAKE ARENA DINING MOON RIVER

In [5]:
import pandas as pd
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# CSVファイルを読み込む
try:
    df = pd.read_csv('/Users/dangararara/lecture/miraisouzou/csv/odaiba_reviews.csv')
except FileNotFoundError:
    print("エラー: 'odaiba_reviews.csv'が見つかりません。")
    df = pd.DataFrame()

# JanomeのTokenizerを準備
t = Tokenizer()

# ストップワード
stop_words = [
    'こと', 'もの', 'それ', 'これ', 'さん', 'よう', 'ため', '的', '的', '気', '感',
    'いる', 'する', 'なる', 'ある', 'ない', 'くる', 'いく', '思う', '言う',
    '美味しい', 'うまい', '普通', '良い', '悪い', '最高', '本当', '今回', '前回', 
    '初', '再訪', '人気', '有名', '定番', '満足', '好き', 'おすすめ', 'ごちそうさま', '様',
    '料理', '食事', 'メニュー', 'ドリンク', 'デザート', '一品', 'ご飯', 'ビール',  '肉', '魚', '野菜',
    '食べる', '飲む', '行く', '来る', '入る', '出る', '頼む', '注文', 
    '訪問', '利用', '頂く', 'いただく', '会計',
    '時間', '日', '時', '分', 
    'お店', 'レストラン', 'こちら', 'ところ', '場所', '雰囲気', '店内', 
    '外観', '席', 'スタッフ', '店員', 
    '自分', '私', '人',
    '一つ', 'たくさん', '少し', 'ちょっと', 'かなり', '非常', 'いろいろ', '色々', 
    '種類', '豊富', '一番', '円', '...',
]


# <--- 変更点：Bi-gram用のTokenizer関数 ---
def tokenize_for_bigrams(text):
    """テキストを形態素解析し、Bi-gramの文字列を返す"""
    # まずは単語に分解
    tokens = []
    for token in t.tokenize(text):
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech in ['名詞', '動詞', '形容詞'] and token.surface not in stop_words:
            tokens.append(token.base_form)
    
    # 単語リストからBi-gramを作成
    bigrams = [f"{tokens[i]}_{tokens[i+1]}" for i in range(len(tokens) - 1)]
    return ' '.join(bigrams)

if not df.empty:
    shop_docs = df.groupby('shop_name')['review_text'].apply(lambda x: ' '.join(x.dropna())).reset_index()
    
    print("各店舗の口コミをBi-gramに変換中...")
    # <--- 変更点：Bi-gram用の関数を適用 ---
    shop_docs['tokens'] = shop_docs['review_text'].apply(tokenize_for_bigrams)
    
    corpus = shop_docs['tokens'].tolist()
    shop_names = shop_docs['shop_name'].tolist()

    if corpus:
        print("Bi-gramのTF-IDFスコアを計算中...")
        vectorizer = TfidfVectorizer(min_df=2, max_df=0.95)
        tfidf_matrix = vectorizer.fit_transform(corpus)
        feature_names = vectorizer.get_feature_names_out()

        print("\n--- 店舗ごとの特徴語（Bi-gramスコア上位）---\n")
        
        for i, shop_name in enumerate(shop_names):
            print(f"【{shop_name}】")
            row = tfidf_matrix.toarray()[i]
            top_n_indices = row.argsort()[::-1][:7]
            
            for index in top_n_indices:
                if row[index] > 0:
                    # <--- 変更点：見やすいようにアンダースコアを置換 ---
                    display_name = feature_names[index].replace('_', ', ')
                    print(f"  - {display_name} (スコア: {row[index]:.3f})")
            print("-" * 20)

各店舗の口コミをBi-gramに変換中...
Bi-gramのTF-IDFスコアを計算中...

--- 店舗ごとの特徴語（Bi-gramスコア上位）---

【1129 by Ogawa】
  - レインボー, ブリッジ (スコア: 0.259)
  - デックス, 東京 (スコア: 0.225)
  - 東京, ビーチ (スコア: 0.184)
  - ハンバーグ, ランチ (スコア: 0.179)
  - ビーチ, 6 (スコア: 0.171)
  - 6, 階 (スコア: 0.159)
  - 台場, 海浜 (スコア: 0.119)
--------------------
【82 築地店】
  - 築地, 店 (スコア: 0.289)
  - 築地, 駅 (スコア: 0.251)
  - ８, ２ (スコア: 0.185)
  - ２, 築地 (スコア: 0.176)
  - 店, hub (スコア: 0.138)
  - 東, 銀座 (スコア: 0.135)
  - メトロ, 日比谷線 (スコア: 0.106)
--------------------
【ABURI 百貫 有明ガーデン店】
  - 回転, 寿司 (スコア: 0.623)
  - 有明, ガーデン (スコア: 0.316)
  - 百, 貫 (スコア: 0.226)
  - 炙る, 寿司 (スコア: 0.159)
  - 寿司, 屋 (スコア: 0.153)
  - 通り, 炙る (スコア: 0.146)
  - 塩, レモン (スコア: 0.115)
--------------------
【ARIAKE ARENA CAFE】
  - 有明, アリーナ (スコア: 0.556)
  - アリーナ, 行く (スコア: 0.163)
  - アリーナ, イベント (スコア: 0.131)
  - ライブ, 有明 (スコア: 0.131)
  - 参加, する (スコア: 0.127)
  - ライブ, 前 (スコア: 0.089)
  - バスケ, 試合 (スコア: 0.081)
--------------------
【ARIAKE ARENA DINING MOON RIVER】
  - 有明, アリーナ (スコア: 0.384)
  - ハンバーガー, ワッフル (スコア: 0.